In [32]:
# import Stemmer
import re
import bisect
import xml.sax
import timeit
import subprocess
import mwparserfromhell
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import Stemmer
import re
import math
import timeit
stop_words = set(stopwords.words('english'))
stemmer = Stemmer.Stemmer('english')

reg1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
reg2 = re.compile(r'{\|(.*?)\|}',re.DOTALL)
reg3 = re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
refreg=r'{{v?cite(.*?)}}'
reg4 = re.compile(r'[-.,:;_?()"/\']',re.DOTALL)
reg5 = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
# reg6 = re.compile(r'[\'~` \n\"_!=@#$%-^*+{\[}\]\|\\<>/?]',re.DOTALL)
reg6 = re.compile(r'[\'~`\"_!=@#$%\-^*+{\[}\]\|\\<>/?]',re.DOTALL)

catRegExp = r'\[\[category:(.*?)\]\]'
infoRegExp = r'{{infobox(.*?)}}'
refRegExp = r'== ?references ?==(.*?)=='
reg7 = re.compile(infoRegExp,re.DOTALL)
reg8 = re.compile(refRegExp,re.DOTALL)
reg9 = re.compile(r'{{(.*?)}}',re.DOTALL)
reg10 = re.compile(r'<(.*?)>',re.DOTALL)
reg11=re.compile(r'\[\[(.*?)\]\]')
reg12=re.compile(catRegExp,re.DOTALL)

index_path="destination/"

def gettitles(toplist):
    top_titles=[]
    for docid in toplist:
#         print(docid)
        title_file_no=math.floor(docid/30000)
        print(title_file_no,"title_file_no")
        title_index=docid%30000
        print(title_index,"title_index")
        fp=open("title/"+str(title_file_no),'r')
#         fp=open("title/"+str(title_file_no),'r')
        
        i=0
        for line in fp.readlines():
            if(i==title_index):
#                 print('')
                top_titles.append(line[:-1])
            i+=1
    return top_titles
        
        
        
        
    

 
def convert(titledict):
    resdict={}
    for key,val in titledict.items():
        val=val.strip()
        token=val.split()
        token=stemmer.stemWords(token)
        for t in token:
           
            resdict[t]=key
    return resdict

def getposting(index_path,word):
    print("enter getposting")
#     print(word)
    secondaryindex_fp=open("secondary.txt",'r')
    secondary_list=[]
    for line in secondaryindex_fp.readlines():
        secondary_list.append(line[:-1])
        
    index_no=bisect.bisect(secondary_list, word)-1 #index number fetched from secondary index
    print(index_no)
    if index_no==-1:                  #invalid index number
        return ""
    index_path=index_path+str(index_no)+".txt"   # path to index file to be searched
    index_fp=open(index_path,'r')
    
    for entry in index_fp.readlines():
        idx=entry.index('#')
        w=entry[:idx]
#         print(w)
        if(w==word):
            print(word)
#             print(entry)
            return entry[idx:]
    return ""                                   # if posting doesn't exist return empty string

def gettopten(docid_freq_tfidf,top_k):
    rank_list=[]
    top_ten=[]
    
    for docid in docid_freq_tfidf:
#         print(docid)
        rank_list.append([[docid_freq_tfidf[docid][0],docid_freq_tfidf[docid][1]],docid])
        
    rank_list.sort(reverse=True)
#     print(rank_list)
    
    i=0
    for item in rank_list:
        top_ten.append(item[1])
        i+=1
        if i == top_k:
            break
            
    return top_ten
        
        
        
        
        
        
    
    
   
    
        
        
    


# index_path="inverted_index/merged_path/"

# index_path="inverted_index/merged_path/"



def top_search1(st,top_k):
    print("in search")

    st=st.lower()
    index=st.find(':')
    docid_freq_tfidf={}  # dict keys are doc_id and value is list of frequescy and tfidf weight
    if(index>=0):
        print("field query")
        titledict={}

        field='#' 
        buffer=""
        for i in range(len(st)):
            if st[i]==':':
                if field!='#':
                    titledict[field]=buffer[0:len(buffer)-1]
                    buffer=""
                field=st[i-1]
                buffer=""

            else:
                buffer=buffer+st[i]

        if field!='#':
            titledict[field]=buffer
        print(titledict)
        resdict=convert(titledict)     #dict of word and their field type
#         print(resdict)
        print(resdict)

        for word,field in resdict.items():
#             print(word,field,"#####################################################################################################################################################################################################################################################3")
            postings=getposting(index_path,word)
            print(len(postings))
    #         print(postings)

            if postings=="":
                print("posting not found")
                continue


            postings=postings[:-1]  #remove new line

            postings=postings.split('#')
            print(postings[0])
            postings=postings[1:]   #first ele in postings is always "" so remove it
#             print(postings)

            for posting in postings:
            
            
                found=False
                if field=='c':
                    
                    postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                        
                if field=='i':
                    postinglist=re.findall(r'i[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                    
                if field=='t':
                    postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='e':
                    postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='b':
                    postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='r':
                    postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if(found==False):     #field not found so overall tfidf returned
                    postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    else:
                        postinglist=re.findall(r'i[0-9]*',posting,re.DOTALL)
                        if(len(postinglist)):
                            tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        else:
                            postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
                            if(len(postinglist)):
                                tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                            else:
                                postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)
                                if(len(postinglist)):
                                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                                else:
                                    postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
                                    if(len(postinglist)):
                                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                                    else:
                                        postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
                                        if(len(postinglist)):
                                            tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    
                    
#                     poslist=posting.split('@')
#                     print("not_found")
#                     postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
#                     if(len(postinglist)):
#                         tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
#                     tfidf=int(poslist[1])
#                     prepost=poslist[0]

                    docid=re.findall(r'D[0-9]*',posting,re.DOTALL)

                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                    if docid not in docid_freq_tfidf:
                        fre_tfidfsum=[1,tfidf]
                        docid_freq_tfidf[docid]=fre_tfidfsum
                    else:
                        fre_tfidfsum=docid_freq_tfidf[docid]
                        fre_tfidfsum[0]+=1
                        fre_tfidfsum[1]+=tfidf
                        docid_freq_tfidf[docid]=fre_tfidfsum
                else:                #field found
#                     poslist=posting.split('@')
#                     tfidf=int(poslist[1])
#                     prepost=poslist[0]

                    docid=re.findall(r'D[0-9]*',posting,re.DOTALL)

                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                    if docid not in docid_freq_tfidf:
                        fre_tfidfsum=[1,tfidf]
                        docid_freq_tfidf[docid]=fre_tfidfsum
                    else:
                        fre_tfidfsum=docid_freq_tfidf[docid]
                        fre_tfidfsum[0]+=1
                        fre_tfidfsum[1]+=tfidf
                        docid_freq_tfidf[docid]=fre_tfidfsum




        top_ten=gettopten(docid_freq_tfidf,top_k)
        top_titles=gettitles(top_ten)
        print(len(top_titles))
#         print(top_ten)
#         print(top_titles)

    else:
        st=reg1.sub(' ',st)
        st=reg2.sub(' ',st)
        st=reg3.sub(' ',st)
        st=reg4.sub(' ',st)
        st=reg5.sub(' ',st)
        st=reg6.sub(' ',st)
        st=st.strip()
        st=st.split()

        st=[word for word in st if word not in stop_words]
        st=stemmer.stemWords(st)

        for word in st:
#             print(word)
            postings=getposting(index_path,word)
    #         if word.isnumeric():
    #             print(postings)
            if postings=="":
                continue


            postings=postings[:-1]  #remove new line

            postings=postings.split('#')
            postings=postings[1:] 

            for posting in postings:
                postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
                if(len(postinglist)):
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                else:
                    postinglist=re.findall(r'i[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    else:
                        postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
                        if(len(postinglist)):
                            tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        else:
                            postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)
                            if(len(postinglist)):
                                tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                            else:
                                postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
                                if(len(postinglist)):
                                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                                else:
                                    postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
                                    if(len(postinglist)):
                                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
    #             if field=='c':
    #                 postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
    #             if field=='i':
    #                 postinglist=re.findall(r'i[0-9]+',posting,re.DOTALL)
    #             if field=='t':
    #                 postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
    #             if field=='e':
    #                 postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
    #             if field=='b':
    #                 postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
    #             if field=='r':
    #                 postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)

#                 poslist=posting.split('@')
#                 print(posting[0])
#                 tfidf=int(poslist[1])-10
#                 prepost=poslist[0]
    


                docid=re.findall(r'D[0-9]*',posting,re.DOTALL)
                docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                if docid not in docid_freq_tfidf:
                    fre_tfidfsum=[1,tfidf]
                    docid_freq_tfidf[docid]=fre_tfidfsum
                else:
                    fre_tfidfsum=docid_freq_tfidf[docid]
                    fre_tfidfsum[0]+=1
                    fre_tfidfsum[1]+=tfidf
                    docid_freq_tfidf[docid]=fre_tfidfsum

        top_ten=gettopten(docid_freq_tfidf,top_k)

        top_titles=gettitles(top_ten)
        
        
        
    return (top_ten,top_titles)
            
            
def main():

    query_in=open("queries.txt",'r')
    query_out=open("queries_op.txt",'w')

    for line in query_in.readlines():
        print(line)
        line=line.split(',')
        top_k=int(line[0])
        query=line[1][:-1]

        start = timeit.default_timer()
        print("enter search")
        ids_titles=top_search1(query,top_k)         #main search logic
        print(ids_titles,"##########################################################3")
        print(len(ids_titles[0]),"0")
        print(len(ids_titles[1]),"1")
        end = timeit.default_timer()

        for n in range(top_k):
            if n >= len(ids_titles[0]):       #if ids_titles doesn't have the number of documents we are asking for
                break
#             print(ids_titles[0],"id_titles")
            write_line=str(ids_titles[0][n])+","+str(ids_titles[1][n])+"\n"
            query_out.write(write_line)

        t=end-start
        query_out.write(str(round(t,3))+","+str(round(t/top_k,3))+"\n")
        query_out.write("\n")

    query_out.close()
    print("End of processing")

if __name__ == "__main__": 
	main() 
        
        
#     query_out.write("\n")
    
        
    
    

           

                    
                    
                    
                    
                    
                    
    
                    
                    
                    
                
                
                
                
                
                
                
                
                
        
            
                
            
            
            
        
        
        
            
            
        
    

        

# else:
#     st=st.strip()
#     tokens=st.split()
#     tokens=stemmer.stemWords(tokens)
#     fp=open('demo.txt','r')
#     lines=fp.readlines()
#     for line in lines:
#         idx=line.find('#')
#         word=line[0:idx]
#         if word in tokens:
#             print(line)
            

        




            

20, b:Californication Season

enter search
in search
field query
{'b': 'californication season'}
{'californ': 'b', 'season': 'b'}
enter getposting
179
californ
6684

enter getposting
325
season
6175443

216 title_file_no
26345 title_index
72 title_file_no
16917 title_index
144 title_file_no
2342 title_index
282 title_file_no
11970 title_index
206 title_file_no
12257 title_index
326 title_file_no
12585 title_index
22 title_file_no
10110 title_index
207 title_file_no
16898 title_index
185 title_file_no
23642 title_index
84 title_file_no
1765 title_index
57 title_file_no
6786 title_index
90 title_file_no
29820 title_index
233 title_file_no
25268 title_index
56 title_file_no
18655 title_index
34 title_file_no
22196 title_index
44 title_file_no
24968 title_index
81 title_file_no
26386 title_index
324 title_file_no
2524 title_index
172 title_file_no
23438 title_index
186 title_file_no
18672 title_index
20
([6506345, 2176917, 4322342, 8471970, 6192257, 9792585, 670110, 6226898, 5573642, 25217

0
posting not found
enter getposting
254
0
posting not found
enter getposting
349
torr
290019

319 title_file_no
3946 title_index
313 title_file_no
1993 title_index
312 title_file_no
11760 title_index
311 title_file_no
3190 title_index
310 title_file_no
15800 title_index
305 title_file_no
10829 title_index
305 title_file_no
10822 title_index
301 title_file_no
3414 title_index
301 title_file_no
3184 title_index
286 title_file_no
19494 title_index
10
([9573946, 9391993, 9371760, 9333190, 9315800, 9160829, 9160822, 9033414, 9033184, 8599494], ['Nicolás Torres', 'Torre España', 'Alhaurín de la Torre', 'Îles Torres', 'Torre de Belém', 'J Torres', 'J. Torres', 'José María Torre', 'Josemaría Torre', 'Néstor Torres']) ##########################################################3
10 0
10 1
20, t:Wave S8500, i:bada, c:Mobile phones introduced in 2010

enter search
in search
field query
{'t': 'wave s850'}
{'wave': 't', 's850': 't'}
enter getposting
363
wave
1104390

enter getposting
320
s850
85

29

lov
7997

319 title_file_no
6795 title_index
256 title_file_no
25363 title_index
9 title_file_no
8976 title_index
287 title_file_no
23909 title_index
268 title_file_no
6109 title_index
251 title_file_no
27968 title_index
197 title_file_no
11145 title_index
187 title_file_no
6008 title_index
182 title_file_no
27956 title_index
182 title_file_no
27955 title_index
10
([9576795, 7705363, 278976, 8633909, 8046109, 7557968, 5921145, 5616008, 5487956, 5487955], ['LOV-1', 'Skånske Lov', 'LOV', 'Lov Grover', 'Jyske Lov', 'Zev Lov X', 'LOV E (album)', 'Sara Lov', 'Danske Lov', 'Danske lov']) ##########################################################3
10 0
10 1
3, b:V for Vendetta, i:2006

enter search
in search
field query
{'b': 'v for vendett'}
{'v': 'b', 'for': 'b', 'vendett': 'b'}
enter getposting
356
0
posting not found
enter getposting
214
0
posting not found
enter getposting
358
vendett
200

152 title_file_no
14626 title_index
318 title_file_no
12286 title_index
52 title_file_no
22820 titl

sudebnik
41 title_file_no
9199 title_index
206 title_file_no
22456 title_index
129 title_file_no
18502 title_index
15 title_file_no
14722 title_index
227 title_file_no
9580 title_index
299 title_file_no
25489 title_index
300 title_file_no
23996 title_index
306 title_file_no
14395 title_index
191 title_file_no
6058 title_index
10 title_file_no
8773 title_index
276 title_file_no
21155 title_index
281 title_file_no
16548 title_index
207 title_file_no
28843 title_index
174 title_file_no
12962 title_index
23 title_file_no
8714 title_index
287 title_file_no
27321 title_index
151 title_file_no
9980 title_index
54 title_file_no
12825 title_index
277 title_file_no
12332 title_index
155 title_file_no
3284 title_index
73 title_file_no
1019 title_index
20 title_file_no
2088 title_index
278 title_file_no
5768 title_index
175 title_file_no
19885 title_index
175 title_file_no
12225 title_index
39 title_file_no
28187 title_index
([1239199, 6202456, 3888502, 464722, 6819580, 8995489, 9023996, 9194395, 